In [1]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


import multiprocessing as mp
import argparse

import math
import heapq # for retrieval topK

import json
import csv
import pandas as pd
from tqdm import tqdm  # tqdm 라이브러리 임포트
from tqdm.auto import trange
import random
from multiprocessing import Pool
import _multiprocessing
import pickle
from scipy.spatial.distance import pdist, squareform

In [2]:
def get_data(path = 'dataset/'):
    # 위도 경도 load
    business_info_file = path + 'business_info.csv' # 필라델피아 가게 정보 business_id, latitude, longitude, city, idx
    
    business_location = []
    with open(business_info_file, 'r', newline='') as business_file:
        csv_reader = csv.reader(business_file)
        next(csv_reader)  # 헤더 행 건너뛰기
        for row in csv_reader:
            _, latitude, longitude, _, _ = row[0], row[1], row[2], row[3].lower(), row[4] #city : 소문자로 받음

            business_location.append([latitude, longitude])
    business_location = np.array(business_location, dtype=float)

    # 리뷰 load
    input_file = path + "reviews.txt"
    data = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            data.append(row)

    if not data:  # 데이터가 비어있는 경우 처리
        return

    user_history_list = []
    user_reviews_list = []
    user_ratings_list = []
    user_review_emb_list = []

    tmp_reviews = []
    tmp_ratings = []
    tmp_business_id = []
    tmp_review_emb_list = []

    before_user_id = data[0][0]  # 첫 번째 사용자 ID로 초기화
    for idx, i in enumerate(data):
        user_id, business_id, rating, review = i
        if user_id == before_user_id:
            tmp_business_id.append(int(business_id))
            tmp_ratings.append(float(rating))
            tmp_reviews.append(review)
            tmp_review_emb_list.append(idx)
        else:
            if len(tmp_business_id) >= 10:  # 방문 횟수가 10회가 넘는 유저만 append
                user_history_list.append(tmp_business_id)
                user_ratings_list.append(tmp_ratings)
                user_reviews_list.append(tmp_reviews)
                user_review_emb_list.append(tmp_review_emb_list)
            tmp_business_id = [int(business_id)]
            tmp_ratings = [float(rating)]
            tmp_reviews = [review]
            tmp_review_emb_list = [idx]

            before_user_id = user_id  # 현재 사용자 ID로 업데이트

    # 마지막 사용자 처리
    if len(tmp_business_id) >= 10:
        user_history_list.append(tmp_business_id)
        user_ratings_list.append(tmp_ratings)
        user_reviews_list.append(tmp_reviews)
        user_review_emb_list.append(tmp_review_emb_list)
    print(len(user_history_list), len(user_ratings_list), len(user_reviews_list), len(user_review_emb_list))


    # POI가 가진 리뷰 임베딩을 획득하기 위해
    # history_list를 기준으로 POI에 방문한 사람들 list 생성
    poi_visited_list = []
    for user,history in enumerate(user_history_list):
        for idx, poi in enumerate(history):
            poi_visited_list.append([int(user), int(poi), float(user_ratings_list[user][idx]), user_reviews_list[user][idx], user_review_emb_list[user][idx]])

    poi_visited_list.sort(key = lambda x:x[1]) # poi 번호 순으로 정렬

    item_history_list = []
    item_reviews_list = []
    item_ratings_list = []
    item_review_emb_list = []


    tmp_reviews = []
    tmp_ratings = []
    tmp_user_id = []
    tmp_review_emb = []
    before_poi_id = poi_visited_list[0][1]  # 첫 번째 POI ID로 초기화

    for idx, i in enumerate(poi_visited_list):
        user_id, business_id, rating, review, review_emb = i[0], i[1], i[2], i[3], i[4]
        if business_id == before_poi_id: # 이전 POI Id와 동일하다면
            tmp_user_id.append(user_id)
            tmp_ratings.append(rating)
            tmp_reviews.append(review)
            tmp_review_emb.append(review_emb)
        else: # 이전 POI ID와 다른 POI라면
            #print(business_id)
            # 이전 POI 정보 안에 있던거 다 추가하고
            item_history_list.append(tmp_user_id)
            item_ratings_list.append(tmp_ratings)
            item_reviews_list.append(tmp_reviews)
            item_review_emb_list.append(tmp_review_emb)

            if int(business_id) - int(before_poi_id) > 1:
                for _ in range(int(business_id) - int(before_poi_id) - 1):
                    #print(f"방문 기록이 없는 POI는 PASS")
                    item_history_list.append([])
                    item_ratings_list.append([])
                    item_reviews_list.append([])
                    item_review_emb_list.append([])

            tmp_user_id = [user_id]
            tmp_ratings = [rating]
            tmp_reviews = [review]
            tmp_review_emb = [review_emb]

            before_poi_id = business_id  # 현재 사용자 ID로 업데이트

    item_history_list.append(tmp_business_id)
    item_ratings_list.append(tmp_ratings)
    item_reviews_list.append(tmp_reviews)
    item_review_emb_list.append(tmp_review_emb)

    print(len(item_history_list), len(item_ratings_list), len(item_reviews_list), len(item_review_emb_list))

    # 임베딩 load
    embedding_file = path + 'embeddings.npy'
    embeddings = np.load(embedding_file, mmap_mode='r')

    user_review_embs = []
    for poi, embeds in enumerate(user_review_emb_list):
        if len(embeds)>0: # 비어있지 않으면
            new_array = np.array([embeddings[idx] for idx in embeds])
            new_array = np.mean(new_array, axis = 0)
        else:
            new_array = np.zeros(768, dtype=np.float32)
        user_review_embs.append(new_array)

    item_review_embs = []
    for poi, embeds in enumerate(item_review_emb_list):
        if len(embeds)>0: # 비어있지 않으면
            new_array = np.array([embeddings[idx] for idx in embeds])
            new_array = np.mean(new_array, axis = 0)
        else:
            new_array = np.zeros(768, dtype=np.float32)

        item_review_embs.append(new_array.tolist())

    return user_history_list, user_ratings_list, user_reviews_list, user_review_embs, item_history_list, item_ratings_list, item_reviews_list, item_review_embs, business_location

In [3]:
class Yelp(Dataset):
    def __init__(self):
        """
        Yelp 데이터셋을 로드하고 학습 데이터와 테스트 데이터를 생성합니다.

        Args:
            dir (str): 데이터 파일이 있는 디렉토리 경로.
            splitter (str): 파일에서 열을 구분하는 구분자.
            K (int): K 값, 즉 각 사용자마다 테스트에 사용되는 상호작용의 수.
        """
        path = 'dataset/'
        user_history_list, _, _, user_review_embeds ,_,_,_,poi_review_embeds = get_data(path)

        self.train = []
        self.test = []
        self.poi_review_embeds = torch.tensor(poi_review_embeds).to(DEVICE)
        self.user_review_embeds = torch.tensor(user_review_embeds).to(DEVICE)
        self.num_user = len(user_history_list)
        self.num_item = len(poi_review_embeds) # 14585

        items = [i for i in range(self.num_item)]
        self.neg = dict()

        random.seed(30)
        for u, hist in enumerate(user_history_list):
            random.shuffle(hist)
            self.train.append(hist[:int(len(hist) * 0.7)])
            self.test.append(hist[int(len(hist) * 0.7) :])

            u_negs = set(items) - set(hist)
            self.neg[u] = list(u_negs) # ng dataset 생성

        # self.test_for_eval = []
        # for u,hist in enumerate(self.test):
        #     for i in hist:
        #         self.test_for_eval.append([u,i])

        self.index_map = []
        for u, user_items in enumerate(self.train):
            for i in user_items:
                self.index_map.append((u, i))

    def __len__(self):
        """
        데이터셋의 사용자 수를 반환합니다.
        """
        #return self.num_user
        return len(self.index_map)

    def __getitem__(self, idx):
        """
        데이터셋에서 하나의 샘플을 가져옵니다.

        Args:
            idx (int): 데이터셋 내의 인덱스.

        Returns:
            u: 사용자 ID.
            i: 긍정적인 아이템 ID.
            j: 부정적인 아이템 ID.
        """
        # u = idx
        # # 사용자별로 하나의 긍정적인 상호작용 선택
        # i = self.train[u][np.random.randint(0, len(self.train[u]))]
        # # 부정적인 상호작용 무작위 선택
        # j = self.neg[u][np.random.randint(0, len(self.neg[u]))]
        # #j = random.sample(self.neg[u], 4)
        # return (u, i, j)

        u, i = self.index_map[idx]
        # 부정적인 아이템 무작위 선택
        j = self.neg[u][np.random.randint(0, len(self.neg[u]))]
        return (u, i, j)

In [4]:
class MFbpr(nn.Module):
    '''
    MF 모델에 대한 BPR 학습
    '''
    def __init__(self, dataset, factors, learning_rate, reg, init_mean, init_stdev):
        '''
        생성자
        Args:
            dataset: 데이터셋 객체로, 학습 및 테스트 데이터를 포함합니다.
            factors (int): 잠재 요인의 수.
            learning_rate (float): 최적화에 사용되는 학습률.
            reg (float): 정규화 강도.
            init_mean (float): 초기화에 사용되는 정규 분포의 평균.
            init_stdev (float): 초기화에 사용되는 정규 분포의 표준 편차.
        '''
        super(MFbpr, self).__init__()
        self.dataset = dataset
        self.train_data = dataset.train
        self.test_data = dataset.test
        self.num_user = dataset.num_user
        self.num_item = dataset.num_item
        self.neg = dataset.neg
        self.factors = factors
        self.learning_rate = learning_rate
        self.reg = reg
        self.init_mean = init_mean
        self.init_stdev = init_stdev


        # 사용자와 아이템의 잠재 요인을 초기화합니다.
        self.embed_user = torch.normal(mean=self.init_mean * torch.ones(self.num_user, self.factors), std=self.init_stdev).to(DEVICE).requires_grad_()
        self.embed_item = torch.normal(mean=self.init_mean * torch.ones(self.num_item, self.factors), std=self.init_stdev).to(DEVICE).requires_grad_()

        # Adam optimizer를 초기화합니다.
        self.mf_optim = optim.Adam([self.embed_user, self.embed_item], lr=self.learning_rate)

    def forward(self, u, i, j):
        '''
        MF-BPR 모델의 forward pass입니다.
        Args:
            u: 사용자 ID.
            i: 긍정적인 아이템 ID.
            j: 부정적인 아이템 ID.
        Returns:
            y_ui: 사용자와 긍정적인 아이템 간의 예측 점수.
            y_uj: 사용자와 부정적인 아이템 간의 예측 점수.
            loss: BPR 손실.
        '''
        # 사용자와 긍정적인 아이템 간의 예측 점수 계산
        y_ui = (self.embed_user[u] * self.embed_item[i]).sum(dim=-1)
        # 사용자와 부정적인 아이템 간의 예측 점수 계산
        y_uj = (self.embed_user[u] * self.embed_item[j]).sum(dim=-1)
        # 정규화 항 계산
        regularizer = self.reg * (torch.sum(self.embed_user[u] ** 2) + torch.sum(self.embed_item[i] ** 2) + torch.sum(self.embed_item[j] ** 2))
        # BPR 손실 계산
        loss = regularizer - torch.sum(torch.log(torch.sigmoid(y_ui - y_uj)))
        return y_ui, y_uj, loss

    def build_model(self, epoch=30, batch_size=32, topK = 10):
        '''
        MF-BPR 모델을 구축하고 학습합니다.
        Args:
            epoch (int): 학습의 최대 반복 횟수.
            num_thread (int): 병렬 실행을 위한 스레드 수.
            batch_size (int): 학습용 배치 크기.
        '''
        data_loader = DataLoader(self.dataset, batch_size=batch_size)

        print("Training MF-BPR with: learning_rate=%.4f, regularization=%.7f, factors=%d, #epoch=%d, batch_size=%d."
               % (self.learning_rate, self.reg, self.factors, epoch, batch_size))
        t1 = time.time()

        max_hit, max_precision, max_recall, max_recall_epoch, max_precision_epoch, max_hit_epoch = 0,0,0,0,0,0
        for epoc in range(epoch):
            iter_loss = 0
            for s, (users, items_pos, items_neg) in enumerate(data_loader):
                # 기울기 초기화
                self.mf_optim.zero_grad()
                # Forward pass를 통해 예측과 손실 계산
                y_ui, y_uj, loss = self.forward(users, items_pos, items_neg)
                iter_loss += loss
                # Backward pass 및 파라미터 업데이트
                loss.backward()
                self.mf_optim.step()
            t2 = time.time()

            # 성능 측정 함수를 통해 HitRatio 및 NDCG를 계산

            hits, recall, precision = self.evaluate_model(self.test_data, topK)

            print(f"epoch={epoc}, loss = {iter_loss}[{int(t2-t1)}s] HitRatio@{topK} = {hits}, RECAll@{topK} = {recall}, PRECISION@{topK} = {precision} [{int(time.time()-t2)}s]")
            t1 = time.time()
            if precision > max_precision:
                max_precision = precision
                max_precision_epoch = epoc
            if recall > max_recall:
                max_recall = recall
                max_recall_epoch = epoc
            if hits > max_hit:
                max_hit = hits
                max_hit_epoch = epoc
            t1 = time.time()

        #save_perform(reg, batch_size, latent_factors, text_factors, epoc, learning_rate, max_hit, max_hit_epoch, max_recall, max_recall_epoch, max_precision, max_precision_epoch)


    def evaluate_model(self, test, K):
        """
        Top-K 추천의 성능(Hit_Ratio, NDCG)을 평가합니다.
        반환값: 각 테스트 상호작용의 점수.
        """
        score_matrix = torch.mm(self.embed_user, self.embed_item.t())
        top_scores, top_indicies = torch.topk(score_matrix, K, dim=1)

        hits = 0
        sum_recall = 0
        sum_precision = 0
        for u,hist in enumerate(test):
            set_topk = set(i.item() for i in (top_indicies[u]))
            set_hist = set(hist)

            if set_hist & set_topk:
                hits += 1
            sum_precision += len(set_hist & set_topk) / len(set_topk)
            sum_recall += len(set_hist & set_topk) / len(set_hist)

        return hits / len(test), sum_recall / len(test), sum_precision / len(test)


In [6]:
class TextBPR(nn.Module):
    '''
    MF 모델에 대한 BPR 학습
    '''
    def __init__(self, dataset, factors, text_factors, learning_rate, reg, init_mean, init_stdev, alpha):
        '''
        생성자
        Args:
            dataset: 데이터셋 객체로, 학습 및 테스트 데이터를 포함합니다.
            factors (int): 잠재 요인의 수.
            learning_rate (float): 최적화에 사용되는 학습률.
            reg (float): 정규화 강도.
            init_mean (float): 초기화에 사용되는 정규 분포의 평균.
            init_stdev (float): 초기화에 사용되는 정규 분포의 표준 편차.
        '''
        super(TextBPR, self).__init__()
        self.dataset = dataset
        self.train_data = dataset.train
        self.test_data = dataset.test
        # self.test_for_eval = dataset.test_for_eval
        self.num_user = dataset.num_user
        self.num_item = dataset.num_item
        self.neg = dataset.neg
        self.factors = factors
        self.factors_Text = text_factors
        self.learning_rate = learning_rate
        self.reg = reg
        self.init_mean = init_mean
        self.init_stdev = init_stdev
        # self.alpha = alpha
        self.alpha = nn.Parameter(torch.tensor(alpha)).to(DEVICE).requires_grad_()

        self.user_review_embeds = dataset.user_review_embeds.to(DEVICE)
        self.poi_review_embeds = dataset.poi_review_embeds.to(DEVICE)

        # 사용자와 아이템의 잠재 요인을 초기화합니다.
        self.embed_user = torch.normal(mean=self.init_mean * torch.ones(self.num_user, self.factors), std=self.init_stdev).to(DEVICE).requires_grad_()
        self.embed_item = torch.normal(mean=self.init_mean * torch.ones(self.num_item, self.factors), std=self.init_stdev).to(DEVICE).requires_grad_()

        self.beta_items = torch.normal(mean=self.init_mean * torch.ones(self.num_item, 1), std=self.init_stdev).to(DEVICE).requires_grad_()
        self.text_bias = torch.normal(mean=self.init_mean * torch.ones(768, 1), std=self.init_stdev).to(DEVICE).requires_grad_()

        # Adam optimizer를 초기화합니다.
        self.mf_optim = optim.Adam([self.embed_user, self.embed_item, self.beta_items, self.text_bias], lr=self.learning_rate, weight_decay=1e-5)

    def forward(self, u, i, j):
        '''
        MF-BPR 모델의 forward pass입니다.
        Args:
            u: 사용자 ID.
            i: 긍정적인 아이템 ID.
            j: 부정적인 아이템 ID.
        Returns:
            y_ui: 사용자와 긍정적인 아이템 간의 예측 점수.
            y_uj: 사용자와 부정적인 아이템 간의 예측 점수.
            loss: BPR 손실.
        '''
        # 사용자와 긍정적인 아이템 간의 예측 점수 계산

        user_latent_factor = self.embed_user[u]
        user_text_factors = self.user_review_embeds[u] / math.sqrt(786)
        alpha = self.alpha


        i_bias = self.beta_items[i] # batch * 1
        j_bias = self.beta_items[j] # batch * 1

        i_text_factors = self.poi_review_embeds[i] # batch * 768
        j_text_factors = self.poi_review_embeds[j] # batch * 768

        i_latent_factors = self.embed_item[i]
        j_latent_factors = self.embed_item[j]

        diff_latent_factors = i_latent_factors - j_latent_factors # batch * latent
        diff_text_factors = (i_text_factors - j_text_factors) / math.sqrt(768) # batch * 768

        if diff_text_factors.shape[0] == 768: # [768], eval set이라면
            user_latent_factor = user_latent_factor.unsqueeze(0)
            user_text_factors = user_text_factors.unsqueeze(0)
            diff_text_factors = diff_text_factors.unsqueeze(0) # [1, text_emb]
            diff_latent_factors = diff_latent_factors.unsqueeze(0) # [ 1, latent_emb]

        latent_factor = (user_latent_factor * diff_latent_factors).sum(dim=-1).unsqueeze(-1)
        text_factor = (user_text_factors * diff_text_factors).sum(dim=-1).unsqueeze(-1)

        u_i_score = alpha * latent_factor + (1 - alpha) * text_factor
        text_bias = diff_text_factors.mm(self.text_bias)

        x_uij = i_bias - j_bias + u_i_score + text_bias

        # 정규화 항 계산
        # BPR 손실 계산
        loss = -torch.sum(torch.log(torch.sigmoid(x_uij.unsqueeze(0))))
        return loss

    def build_model(self, epoch=30, batch_size=32, topK = 10):
        '''
        MF-BPR 모델을 구축하고 학습합니다.
        Args:
            epoch (int): 학습의 최대 반복 횟수.
            num_thread (int): 병렬 실행을 위한 스레드 수.
            batch_size (int): 학습용 배치 크기.
        '''
        data_loader = DataLoader(self.dataset, batch_size=batch_size)

        print("Training MF-BPR with: learning_rate=%.4f, regularization=%.7f, factors=%d, #epoch=%d, batch_size=%d."
               % (self.learning_rate, self.reg, self.factors, epoch, batch_size))
        t1 = time.time()

        max_hit, max_precision, max_recall, max_recall_epoch, max_precision_epoch, max_hit_epoch = 0,0,0,0,0,0
        for epoc in range(epoch):
            iter_loss = 0
            count = 0
            for s, (users, items_pos, items_neg) in enumerate(data_loader):
                users = users.to(DEVICE)
                items_pos = items_pos.to(DEVICE)
                items_neg = items_neg.to(DEVICE)

                count += 1
                # 기울기 초기화
                self.mf_optim.zero_grad()
                # Forward pass를 통해 예측과 손실 계산
                loss = self.forward(users, items_pos, items_neg)
                iter_loss += loss
                # Backward pass 및 파라미터 업데이트
                loss.backward()
                self.mf_optim.step()
            t2 = time.time()

            # 성능 측정 함수를 통해 HitRatio 및 NDCG를 계산
            hits, recall, precision = self.evaluate_model(self.test_data, topK)
            # eval_loss = 0
            # for idx, (u, i, j) in enumerate(self.test_for_eval):
            #     u, i, j = u.to(DEVICE), i.to(DEVICE), j.to(DEVICE)
            #     loss = self.forward(u, i, j)
            #     eval_loss += loss
            # total_samples = len(self.test_for_eval)
            # eval_loss = eval_loss / total_samples if total_samples > 0 else 0
            # iter_loss = iter_loss / count / batch_size
            print(f"epoch={epoc}, train_loss = {iter_loss:.6} [{int(t2-t1)}s] HitRatio@{topK} = {hits:.6}, RECAll@{topK} = {recall:.6}, PRECISION@{topK} = {precision:.6} [{int(time.time()-t2)}s], alpha: {alpha}")
            t1 = time.time()
            if precision > max_precision:
                max_precision = precision
                max_precision_epoch = epoc
            if recall > max_recall:
                max_recall = recall
                max_recall_epoch = epoc
            if hits > max_hit:
                max_hit = hits
                max_hit_epoch = epoc
            t1 = time.time()

        #save_perform(reg, batch_size, latent_factors, text_factors, epoc, learning_rate, max_hit, max_hit_epoch, max_recall, max_recall_epoch, max_precision, max_precision_epoch, alpha)


    def evaluate_model(self, test, K):
        """
        Top-K 추천의 성능(Hit_Ratio, NDCG)을 평가합니다.
        반환값: 각 테스트 상호작용의 점수.
        """
        user_latent_factor = self.embed_user # batch * latent
        item_latent_factors = self.embed_item # batch * latent

        user_text_factors = self.user_review_embeds / math.sqrt(768) # batch * latent
        item_text_factors = self.poi_review_embeds / math.sqrt(768)# batch * 768


        latent_score_matrix = torch.mm(user_latent_factor, item_latent_factors.t())
        text_score_matrix = torch.mm(user_text_factors, item_text_factors.t())

        score_matrix = self.alpha * latent_score_matrix + (1-self.alpha) * text_score_matrix

        item_bias = self.beta_items.squeeze()
        item_bias = item_bias.view(1, -1)
        score_matrix = score_matrix + item_bias


        top_scores, top_indicies = torch.topk(score_matrix, K, dim=1)

        hits = 0
        sum_recall = 0
        sum_precision = 0
        for u,hist in enumerate(test):
            set_topk = set(i.item() for i in (top_indicies[u]))
            set_hist = set(hist)

            if set_hist & set_topk:
                hits += 1
            sum_precision += len(set_hist & set_topk) / len(set_topk)
            sum_recall += len(set_hist & set_topk) / len(set_hist)

        return hits / len(test), sum_recall / len(test), sum_precision / len(test)


In [7]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
np.random.seed(30)
yelp = Yelp()
# with open('/content/drive/MyDrive/학교/졸업작품/yelp.pkl', 'wb') as f:
#     pickle.dump(yelp, f)

In [10]:
# with open('/content/drive/MyDrive/학교/졸업작품/yelp.pkl', 'rb') as f:
#     yelp = pickle.load(f)

In [11]:
# 개선 한 유저당 n개의 긍정, 부정 데이터셋(n = 방문한 모든 POI)
len(yelp)

319337

In [13]:
if __name__ == '__main__':
    np.random.seed(30)  # 시드 설정을 통해 일관된 결과를 얻기 위해

    #yelp = Yelp()

    latent_factors = 768  # 잠재요인 수
    text_factors = 768  # 텍스트 잠재요인 수
    learning_rate = 0.0005  # 학습률
    reg = 1e-4  # 정규화 계수
    init_mean = 0  # 초기 가중치 평균
    init_stdev = 0.001  # 초기 가중치 표준편차
    epoch = 100  # 최대 반복 횟수
    batch_size = 1024  # 미니배치 크기
    alpha = 0.75
    num_thread = mp.cpu_count() # 사용할 스레드 수
    print("thread num: ", num_thread)
    K = 10
    print("#factors: %d, lr: %f, reg: %f, batch_size: %d" % (latent_factors, learning_rate, reg, batch_size))

    # MF-BPR 모델 생성 및 학습
    #bpr = MFbpr(yelp, latent_factors, learning_rate, reg, init_mean, init_stdev)
    bpr = TextBPR(yelp, latent_factors, text_factors, learning_rate, reg, init_mean, init_stdev, alpha)

    bpr.build_model(epoch, batch_size=batch_size, topK = K)

    # 학습된 가중치 저장
    # np.save("out/u"+str(learning_rate)+".npy", bpr.U.detach().numpy())
    # np.save("out/v"+str(learning_rate)+".npy", bpr.V.detach().numpy())

thread num:  12
#factors: 768, lr: 0.000500, reg: 0.000100, batch_size: 1024
Training MF-BPR with: learning_rate=0.0005, regularization=0.0001000, factors=768, #epoch=100, batch_size=1024.
epoch=0, loss = 221348.671875[4e+00s] HitRatio@10 = 0.19575456886265152, RECAll@10 = 0.032288273353077344, PRECISION@10 = 0.02319286566601819 [2e+00s]
epoch=1, loss = 212598.328125[4e+00s] HitRatio@10 = 0.22049864975193118, RECAll@10 = 0.035505931570072695, PRECISION@10 = 0.02809772027884306 [2e+00s]
epoch=2, loss = 151832.546875[4e+00s] HitRatio@10 = 0.22087546316648873, RECAll@10 = 0.03547939871918259, PRECISION@10 = 0.028009797148779617 [2e+00s]
epoch=3, loss = 102255.453125[4e+00s] HitRatio@10 = 0.2301073918231489, RECAll@10 = 0.03718448763473138, PRECISION@10 = 0.029247001193243725 [2e+00s]
epoch=4, loss = 89560.328125[6e+00s] HitRatio@10 = 0.23789486905733845, RECAll@10 = 0.03880752950596642, PRECISION@10 = 0.0305721283677712 [2e+00s]
epoch=5, loss = 81207.6875[4e+00s] HitRatio@10 = 0.249073667

KeyboardInterrupt: 